In [1]:
from wildlife_datasets import datasets, splits
import torchvision.transforms as T
import wildlife_tools.data.__init__ as tools
import timm
from wildlife_tools.features import DeepFeatures
from sklearn.metrics import adjusted_rand_score
from active_semi_clustering.semi_supervised.pairwise_constraints import PCKMeans, COPKMeans, MKMeans, MPCKMeans, MPCKMeans, RCAKMeans
from active_semi_clustering.semi_supervised.labeled_data import KMeans
from active_semi_clustering.active.pairwise_constraints import ExampleOracle, ExploreConsolidate, MinMax
from sklearn import metrics

C:\Users\Juanc\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Cargamos el dataset de tortugas
dataset_path = 'data/CowDataset'
datasets.CowDataset.get_data(dataset_path)
metadata = datasets.CowDataset(dataset_path)
data_df = metadata.df

DATASET CowDataset: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [3]:
transform = T.Compose([T.Resize(size=(384, 384)),
                              T.ToTensor(), 
                              T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
dataset = tools.WildlifeDataset(metadata=data_df, root="./data/CowDataset", transform=transform)

In [4]:
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-L-384', pretrained=True)
extractor = DeepFeatures(backbone)
outputFeatures = extractor(dataset)

100%|██████████████████████████████████████████████████████████████| 12/12 [42:58<00:00, 214.88s/it]


In [5]:
outputClasses = data_df['identity'].to_numpy()

In [6]:
# Este es con la S 224
clusterer = PCKMeans(n_clusters=13,max_iter=100, w=0)
clusterer.fit(outputFeatures)
metrics.adjusted_rand_score(outputClasses, clusterer.labels_)

0.4182548587999259